In [1]:
# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs

In [5]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

## Loading the required DataFrame

In [6]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head(12)

,Unnamed: 0,Postal Code,Latitude,Longitude
0,0,M1B,43.806686,-79.194353
1,1,M1C,43.784535,-79.160497
2,2,M1E,43.763573,-79.188711
3,3,M1G,43.770992,-79.216917
4,4,M1H,43.773136,-79.239476
5,5,M1J,43.744734,-79.239476
6,6,M1K,43.727929,-79.262029
7,7,M1L,43.711112,-79.284577
8,8,M1M,43.716316,-79.239476
9,9,M1N,43.692657,-79.264848


## Creating Toronto's Map

In [9]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng in zip(df_toronto['Latitude'], df_toronto['Longitude']):
    label = '{}, {}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

## Create a new neigborhood of places around a given postal code

In [10]:
# @hiddel_cell
CLIENT_ID = 'TP2QRNETE5WCG554R04U3UOG5FCXSVBWR45FNEBCKQZVQVNW' # your Foursquare ID
CLIENT_SECRET = '31ETCRXF32ID1V0DZ1JQZIYCYA3AAIVRMQAGSOI5ADXKXEND' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

In [12]:
postal_code_data = df_toronto[df_toronto['Postal Code'] == 'M1W'].reset_index(drop=True)
postal_code_data

,Unnamed: 0,Postal Code,Latitude,Longitude
0,15,M1W,43.799525,-79.318389


## Create Map

In [14]:
post_code = 'Postal Code,M1W'
latitude_add = 43.799525
longitude_add = -79.318389
print('The geograpical coordinate of Postal Code - M1W are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Postal Code - M1W are 43.799525, -79.318389.


In [16]:
map_scarb = folium.Map(location=[latitude_add, longitude_add], zoom_start=12)

# add markers to map
for lat, lng, label in zip(postal_code_data['Latitude'], postal_code_data['Longitude'], postal_code_data['Postal Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

## Get top 100 nearby venues

In [19]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_add, longitude_add, VERSION, radius, LIMIT)

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f26fccdee0a6101c5b4a119'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': "L'Amoreaux",
  'headerFullLocation': "L'Amoreaux, Toronto",
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 15,
  'suggestedBounds': {'ne': {'lat': 43.804025004500005,
    'lng': -79.3121659267501},
   'sw': {'lat': 43.7950249955, 'lng': -79.3246120732499}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4df28ea0e4cda09e6da0129a',
       'name': 'Mr Congee Chinese Cuisine 龍粥記',
       'location': {'address': '2900 Warden Ave',
        'crossStreet': 'at Finch Ave. E',
        'lat': 43.798878792587615,
        'lng': -79.3183345011537,
        'labeledL